In [ ]:
"""
Created on Wed Aug 09 16:56 2023

Try to infer some Bayesian weights

Author: @claraburgard

"""

In [ ]:
import xarray as xr

In [ ]:
mod = 'CNRM-CM6-1'

In [ ]:
home_path = '/bettik/burgardc/'

In [ ]:
inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)

READ IN CALVING FLUX

In [ ]:
inputpath_caving = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/'
calving_flux = xr.open_dataset(inputpath_caving + 'calving_flux_rignot13.nc')

READ IN ATMO

model

In [ ]:
inputpath_atmo = '/bettik/kittelc/clara/'

scenario = 'historical'
melt_atmo = xr.open_dataset(inputpath_atmo+mod+'_SMB_'+scenario+'.nc').sel(Nisf=rignot_isf)
melt_atmo['time'] = melt_atmo['time'].dt.year

reanalysis

READ IN OCEAN

model

In [ ]:
param_classic_list = ['linear_local',
              'quadratic_local','quadratic_local_locslope',
              'quadratic_mixed_mean','quadratic_mixed_locslope',
              'lazero19',
              'boxes_4_pismyes_picopno']

param_NN_list = ['NNxsmall96',
              'NNsmall','NNlarge']

In [ ]:
## Melt outputpath
outputpath_melt = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/OCEAN_MELT_RATE_CMIP/'+mod+'/'

melt1D_list = []
for mparam in param_classic_list:
    melt1D_scenario = xr.open_dataset(outputpath_melt + 'eval_metrics_1D_'+mparam+'_'+scenario+'_oneFRIS.nc')
    melt1D_list.append(melt1D_scenario.assign_coords({'param':mparam}))
melt1D_classic = xr.concat(melt1D_list, dim='param')       
Gt_classic = melt1D_classic['melt_1D_Gt_per_y']
box1_classic = melt1D_classic['melt_1D_mean_myr_box1']

melt1D_list = []
for mparam in param_NN_list:
    melt1D_scenario = xr.open_dataset(outputpath_melt + 'evalmetrics_1D_'+mparam+'_'+scenario+'.nc')
    melt1D_list.append(melt1D_scenario.assign_coords({'param':mparam}))
melt1D_NN = xr.concat(melt1D_list, dim='param')   
Gt_NN = melt1D_NN['predicted_melt'].sel(metrics='Gt')
box1_NN = melt1D_NN['predicted_melt'].sel(metrics='box1')

Gt_all = xr.concat([Gt_classic, Gt_NN], dim='param')
box1_all = xr.concat([box1_classic, box1_NN], dim='param')

In [ ]:
mass_balance = Gt_all - melt_atmo['SMB'] + calving_flux

obs estimate

In [ ]:
inputpath_Paolo = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/'
data_Paolo = xr.open_dataset(inputpath_Paolo + 'ANT_G1920V01_IceShelfMelt.nc')

In [ ]:
data_Paolo['melt_mean']

In [ ]:
data_Paolo.time.dt.year in range(1992,2015)

In [ ]:
data_Paolo.time.dt.year < 2015